# Quiz Notebook 1: FEI Protocol Model Basics

This Quiz Notebook is associated to [**Analysis Notebook 1: Sanity Checks**](../1_sanity_checks.ipynb).

The goal of this notebook is understanding how to practically make changes and additions to the FEI Protocol radCAD model.

# Quiz Setup 

- The first section of all quiz notebooks contains all necessary code to run the relevant simulation **within the quiz**.
- However, this is done for enabling experimentation. **In practice, each notebook should be gone through with the associated analysis notebook**.
- Analysis notebook sections relevant to quiz questions will be pointed to in the analysis notebooks directly.
- Solution cells are not meant to directly execute, as these may only contain the relevant snippets.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

In [ ]:
import plotly.io as pio
png_renderer = pio.renderers["png"]
png_renderer.width = 1200
png_renderer.height = 500
# png_renderer.scale = 1

pio.renderers.default = "png"

In [ ]:
from operator import lt, gt

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

In [ ]:
# Inspect experiment template
display_code(default_experiment)

### Simulation 1 - Goal: Extend Simulation with Sweep

In [ ]:
# Create a simulation for each analysis
simulation_1 = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
# Analysis-specific setup
parameter_overrides = {
    # Disable policy by setting to `None`
    "target_stable_pcv_ratio": [None],
    "target_stable_backing_ratio": [None],
    "volatile_asset_price_process": [
        lambda _run, _timestep: 2_000,
        lambda _run, timestep: 2_000 if timestep < 365 / 4 else (1_000 if timestep < 365 * 3/4 else 2_000),
        lambda _run, timestep: 2_000 * (1 + timestep * 0.2 / 365),
        lambda _run, timestep: 2_000 * (1 - timestep * 0.2 / 365),
    ],
}

In [ ]:
# Experiment configuration
simulation_1.model.runs = 1

# Override inital state
simulation_1.model.initial_state.update({})

# Override default experiment System Parameters
simulation_1.model.params.update(parameter_overrides)

In [ ]:
# Analysis-specific setup

In [ ]:
# Experiment execution
df, exceptions = run(simulation_1)

In [ ]:
fig = df.plot(y='volatile_asset_price', color='subset')

fig.update_layout(
    title="Volatile Asset (e.g. ETH) Price Trajectories",
    xaxis_title="Timestamp",
    yaxis_title="Volatile Asset Price (USD)",
    autosize=False,
    width=1200,
    height=675,
)

### Part 1 - Sanity Checks Analysis 1

1. Write a **lambda function** for a volatile asset price trajectory of your choosing

In [ ]:
new_trajectory = lambda _run, _timestep: 1_800

2. Extend simulation 1 by adding the new trajectory to the parameter sweep

3. In the **Sanity Checks Analysis Notebook**, add this new trajectory to the **volatile_asset_price_process** parameter sweep and re-execute **Analysis 1** in the notebook. Confirm that you can see the new trajectory as a new element in all relevant plots. Note the dynamics you observe for the price process you added.

### Part 2 - Goal: Extend Simulation with model update

1. How is a new state variable added to a radCAD model?


**A:** A new state variable is added an initialized in the **/model/state_variables.py** system parameters dataclass.

2. Add a state variable named <b>reserve_ratio</b> to the relevant file for setting state variables, and initialize its value to 1


3. Where in the directory structure can partial state update blocks be added?


**A:** New partial state update blocks are added in the partial state update blocks folder. The **/model/parts** folder is used to add in modular policies and state update functions which can be imported into the state update blocks file for execution.

4. Create the file <b>/model/parts/quiz_block.py</b>


5. In this file, create a **state update function** which updates the **reserve_ratio** state variable as: **RR = (PCV + Protocol-Owned FEI) / Total FEI Supply.** You are provided with all state variables needed to compute this.

In [ ]:
def update_reserve_ratio(
    params: Parameters, substep, state_history, previous_state, policy_input
) -> float:
    """Update Reserve Ratio
    State update block for update of the protocol reserve ratio
    """

    # Parameters
    dt = params["dt"]

    # State Variables
    run = previous_state["run"]
    timestep = previous_state["timestep"]
    
    total_pcv = previous_state["total_pcv"]
    total_protocol_owned_fei = previous_state["total_protocol_owned_fei"]
    total_fei_supply = previous_state["total_fei_supply"]
    
    #RR = (PCV + Protocol-Owned FEI) / Total FEI Supply.
    reserve_ratio = (total_pcv + total_protocol_owned_fei) / total_fei_supply
    
    return "reserve_ratio", reserve_ratio


6. How and where do you include a State Update Function as part of a radCAD model?

**A:** a PSUB dict must be constructed and appended to the list of PSUBs in **/model/state_update_blocks.py**

7. In the file **/model/state_update_blocks.py** create a new PSUB (by copying the existing structure) and append it as the last element of the list.

state_update_blocks = [...] #add dict to last element of this list

8. Process the state variable update to **reserve_ratio** through importing and including the state update function you just created in **/model/parts/quiz_block.py**

In [ ]:
#import model.parts.reserve_ratio as reserve_ratio
# within state_update_blocks = [...]
PSUB = {
    description: """
        Update Reserve Ratio Toy PSUB
    """,
    policies: {},
    variables: {
        "reserve_ratio": reserve_ratio.update_reserve_ratio,
    },
},

9. Re-run the **Sanity Checks Analysis Notebook**, and plot the **reserve_ratio** state variable alongside collateral ratio, for a *single* subset, where indicated.

In [ ]:
df_2.query("subset==0").plot(
    x="timestep",
    y=["collateralization_ratio", "reserve_ratio"],
)

10. In the **Sanity Checks Analysis Notebook**, where indicated, add this variable to the **variables** python list, and re-execute notebook Analyses 2.1 and 2.2.

11. What is the correlation between **reserve_ratio** and **collateralization_ratio** ?

**A:** The correlation is of **0.997** - this is expected given both these KPIs encode very similar information.

### Part 3 - Goal: Extend Simulation with model update refactor

We can generally be much more efficient than creating a PSUB with associated model part just for a **single** state variable. Here we show how to add **reserve_ratio** into an existing policy.

1. Remove the existing PSUB from the model by **commenting out** its inclusion in  **/model/state_update_blocks.py**

2. (Optional) Remove the file **/model/parts/quiz_block.py**

3. Which **model part file** is it most appropriate to add in the calculation of this metric: _Hint: see radCAD Differential Spec_

4. Create the calculation for the **reserve_ratio** as part of the appropriate policy function, and add it as an element of the python dict returned by the policy

5. How do you make sure a state update of a variable just added to a policy function is included in radCAD execution?

**A:** you must make sure it is present in the **variables: {}** item of the concerned PSUB

6. How do you update the **reserve_ratio** variable in the PSUB whose description field is “System Metrics”, **without using a state update function?**

**A:** use the radCAD **update_from_signal()** helper

7. Include **reserve_ratio** in the list comprehension for the system metrics PSUB.

PSUB = {
    description: """
        System Metrics
    """,
    policies: {
        "system_metrics": system_metrics.policy_system_metrics,
    },
    variables: {
        key: update_from_signal(key)
        for key in [
            # PCV System Metrics
            "stable_backing_ratio",
            "stable_pcv_ratio",
            "collateralization_ratio",
            "pcv_yield_rate",
            "reserve_ratio", # <-- ADD HERE
            # Protocol System Metrics
            "protocol_equity",
            "protocol_revenue",
        ]
    },
},

8. Re-run the sanity checks notebook and make sure outputs are the same as in the previous set of questions. 